# 1. GEE Image Fusion : MODIS & Landsat 8 for NDVI dense time series calculation

Author: Morgane Magnier (morgane.magnier@vattenfall.com)

Copyright © [2024] [Magnier Morgane]  

This notebook is part of a thesis project. The copyright of the thesis itself belongs to the student Morgane Magnier.  

**Rights and Intellectual Property**:  
- Vattenfall has the right to use the findings, methods, and conclusions of this thesis in its operations.  
- Any material generated within the framework of this thesis that is subject to intellectual property protection 
  (e.g., source code, computer program, design, or invention) belongs to Vattenfall, unless otherwise agreed in writing.  

Permission is granted to view, copy, and share this notebook for **educational or personal purposes only**, provided that this 
notice is included in all copies.  

---

In [64]:
import ee, geemap, eemont
from GEE_ImageFusion import *

ee.Initialize()
ee.Authenticate()

True

## Globals

In [105]:
# region of interest
# location for prediction (outside scene overlap areas)
region = ee.Geometry.Polygon([[[17.204933,60.402663],[17.204933,60.455525],[17.2645,60.455525],[17.2645,60.402663],[17.204933,60.402663]]])

# define training data temporal bounds broadly
startDate = '2013-01-01'
endDate = '2014-01-01'

# common bands between sensors that will be used for fusion
#   would need to add functions for other indices (evi etc.).
#   ndvi is exported as 16 bit int so would have to make sure not to rescale
#   reflectance bands if that is what you are planning to predict (line 206)
#   some resturcturing of this code would be necessary if the goal was to
#   predict a multiband image but the core functions should work for any number
#   of bands
commonBandNames = ee.List(['ndvi'])

# image collections to use in fusion
# NOTE: if using older Landsat and not using NDVI one would have to modify the
# get_paired_collections script because this script harmonizes NDVI from
# L5 & L7 to L8 based on Roy et al. 2016 (see etmToOli and getPaired functions)
landsatCollection = 'LANDSAT/LC08/C02/T1_L2'
modisCollection = 'MODIS/006/MCD43A4'

# landsat band names including qc band for masking
bandNamesLandsat = ee.List(['blue', 'green', 'red',
                            'nir', 'swir1', 'swir2', 'pixel_qa'])
landsatBands = ee.List([1, 2, 3, 4, 5, 6, 10])

# modis band names
bandNamesModis = ee.List(['blue', 'green', 'red', 'nir', 'swir1', 'swir2'])
modisBands = ee.List([2, 3, 0, 1, 5, 6])

# radius of moving window
# Note: Generally, larger windows are better but as the window size increases,
# so does the memory requirement and we quickly will surpass the memory
# capacity of a single node (in testing 13 was max size for single band, and
# 10 was max size for up to 6 bands)
kernelRadius = ee.Number(10)
kernel = ee.Kernel.square(kernelRadius)
numPixels = kernelRadius.add(kernelRadius.add(1)).pow(2)

# number of land cover classes in scene
coverClasses = 7
# to export the images to an asset we need the path to the assets folder
path = 'projects/ee-magniermorgane/assets/'
scene_name = 'NDVI_2013_'


## Get filtered collections

In [84]:
def filter_col(col, roi, band, thresh):
    
    col = col.map(lambda image : image.clip(roi))

    def count_pixels(image,roi): 
        pixel_count = image.select(band).reduceRegion(
            reducer= ee.Reducer.count(),
            geometry=roi,
            scale=10,
            maxPixels=1e9
        ).get(band)
        return image.set('pixel_count', pixel_count)

    nb_pixels_ts = col.map(lambda image: count_pixels(image, roi))

    # Get the image with the maximum pixel count
    max_pixel_count_image = nb_pixels_ts.sort('pixel_count', False).first()
    ref_img_pixel_count = max_pixel_count_image.get('pixel_count').getInfo()
    pixel_count_threshold = ref_img_pixel_count * thresh

    # Filter the collection based on the pixel count threshold
    filtered_col = nb_pixels_ts.filter(ee.Filter.gte('pixel_count', pixel_count_threshold))

    return filtered_col

def getPaired(startDate, endDate,
              landsatCollection, landsatBands, bandNamesLandsat,
              modisCollection, modisBands, bandNamesModis,
              commonBandNames,
              region):
    """
    Create a list of image collections. Landsat and MODIS with low cloud cover\
    from the same date and the MODIS images between these pairs.

    Parameters
    ----------
    startDate: str
        Start date of fusion timeframe.
    endDate: str
        End date of the fusion timeframe.
    landsatCollection: str
        Landsat collection https://developers.google.com/earth-engine/datasets
    landsatBands: ee_list.List
        List of integers corresponding to Landsat bands.
    bandNamesLandsat: ee_list.List
        List of strings used to rename bands.
    modisCollection: str
        MODIS collection https://developers.google.com/earth-engine/datasets
    modisBands: ee_list.List
        List of integers corresponding to MODIS bands in same order as Landsat.
    bandNamesModis: ee_list.List
        List of strings used to rename bands.
    commonBandNames: ee_list.List
        List of bands to use in fusion. Common to both Landsat and MODIS.
    region: geometry.Geometry
        Location to use in filtering collections. Must not be in scene overlap.

    Returns
    -------
    python list obejct
        Each element in this list is an ee.ImageCollection. The first and \
        second elements are the Landsat occuring on the same date and \
        the last element is the MODIS images between each of the pair \
        dates.

    """
    if landsatCollection == 'LANDSAT/LC08/C02/T1_L2':
        #print('okay')
        # get landsat images
        landsat = ee.ImageCollection(landsatCollection) \
                    .filterDate(startDate, endDate) \
                    .filterBounds(region) \
                    .filterMetadata('CLOUD_COVER', 'less_than', 80) \
                    .select(landsatBands, bandNamesLandsat) \
                    .map(addNDVI) \
                    .map(maskLandsat) \
                    .filterMetadata('CloudSnowMaskedPercent', 'less_than', 50)\
                    .map(lambda image: image \
                         .setMulti({
                             'system:time_start':
                                 ee.Date(image.date().format('y-M-d')) \
                                 .millis(),
                             'DOY': image.date().format('D')
                             })) \
                    .select(commonBandNames)\
                    .map(lambda image : image.clip(region))
        
        landsat = filter_col(landsat, roi, 'ndvi', 0.95)
        
        display(landsat)
    else:
        # get landsat images
        landsat = ee.ImageCollection(landsatCollection) \
                    .filterDate(startDate, endDate) \
                    .filterBounds(region) \
                    .filterMetadata('CLOUD_COVER', 'less_than', 5) \
                    .select(landsatBands, bandNamesLandsat) \
                    .map(addNDVI) \
                    .map(maskLandsat) \
                    .filterMetadata('CloudSnowMaskedPercent', 'less_than', 50)\
                    .map(lambda image: image \
                         .setMulti({
                             'system:time_start':
                                 ee.Date(image.date().format('y-M-d')) \
                                 .millis(),
                             'DOY': image.date().format('D')
                             })) \
                    .select(commonBandNames) \
                    .map(etmToOli)\
                    .map(lambda image : image.clip(region))

    # get modis images
    modis = ee.ImageCollection(modisCollection) \
              .filterDate(startDate, endDate) \
              .select(modisBands, bandNamesModis) \
              .map(addNDVI) \
              .map(maskMODIS) \
              .map(lambda image: image.set('DOY', image.date().format('D'))) \
              .select(commonBandNames)\
              .map(lambda image : image.clip(region))

    modis = filter_col(modis, roi, 'ndvi', 0.95)
    # filter the two collections by the date property
    dayfilter = ee.Filter.equals(leftField='system:time_start',
                                 rightField='system:time_start')

    # define simple join
    pairedJoin = ee.Join.simple()
    # define inverted join to find modis images without landsat pair
    invertedJoin = ee.Join.inverted()

    # create collections of paired landsat and modis images
    landsatPaired = pairedJoin.apply(landsat, modis, dayfilter)
    modisPaired = pairedJoin.apply(modis, landsat, dayfilter)
    modisUnpaired = invertedJoin.apply(modis, landsat, dayfilter)

    return [landsatPaired, modisPaired, modisUnpaired]

In [85]:
# sorted, filtered, paired image retrieval
paired = getPaired(startDate, endDate,
                   landsatCollection, landsatBands, bandNamesLandsat,
                   modisCollection, modisBands, bandNamesModis,
                   commonBandNames, region)

# subs_meta = subs.getInfo()

In [86]:
subs = makeSubcollections(paired)
display(subs)

## Predict 

In [87]:
def processGroup(pair):

    # Group of images to predict
    pair = ee.List(pair)
    landsat_t01 = ee.List(pair.get(0))
    modis_t01 = ee.List(pair.get(1))
    modis_tp = ee.List(pair.get(2))
    pred_group = modis_tp
    # Get the start and end day values and year for the group to use
    # to label the file when exported to asset
    startDay = ee.Number.parse(ee.ImageCollection(pred_group)
                               .first()
                               .get('DOY'))
    endDay = ee.Number.parse(ee.ImageCollection(pred_group)
                               .sort('system:time_start', False)
                               .first()
                               .get('DOY'))
    year = ee.Date(ee.ImageCollection(pred_group)
                   .sort('system:time_start', False)
                   .first()
                   .get('system:time_start')).format('Y')

    # Start and end day of year
    doys = landsat_t01 \
        .map(lambda img: ee.String(ee.Image(img).get('DOY')).cat('_'))

    # Register images
    landsat_t01, modis_t01, modis_tp = registerImages(landsat_t01,
                                                      modis_t01,
                                                      modis_tp)

    # Prep Landsat imagery (mask and format)
    maskedLandsat, pixPositions, pixBN = prepLandsat(landsat_t01,
                                                     kernel,
                                                     numPixels,
                                                     commonBandNames,
                                                     doys,
                                                     coverClasses)
    
    # Prep MODIS imagery (mask and format)
    modSorted_t01, modSorted_tp = prepMODIS(modis_t01, modis_tp, kernel,
                                            numPixels, commonBandNames,
                                            pixBN)
    # Calculate spectral distance
    specDist = calcSpecDist(maskedLandsat, modSorted_t01,
                            numPixels, pixPositions)

    # Calculate spatial distance
    spatDist = calcSpatDist(pixPositions)

    # Calculate weights from the spatial and spectral distances
    weights = calcWeight(spatDist, specDist)

    # Calculate the conversion coefficients
    coeffs = calcConversionCoeff(maskedLandsat, modSorted_t01,
                                 doys, numPixels, commonBandNames)

    # Predict all MODIS images in the modis tp collection and retain the date
    
    predictions = modSorted_tp.map(lambda image: predictLandsat(landsat_t01, modSorted_t01, doys, ee.List(image), weights, coeffs, commonBandNames, numPixels))

    return ee.ImageCollection(predictions)


In [88]:
total_size = subs.size().getInfo()

# Vérifier si le nombre de paires est supérieur à 4
if total_size > 4:
    # Déterminer la taille de chaque partition
    partition_size = total_size // 4

    # Diviser subcols en 4 parties
    subcols_part1 = subs.slice(0, partition_size)
    subcols_part2 = subs.slice(partition_size, 2 * partition_size)
    subcols_part3 = subs.slice(2 * partition_size, 3 * partition_size)
    subcols_part4 = subs.slice(3 * partition_size, total_size)

    # Appliquer processGroup à chaque partie
    result_part1 = subcols_part1.map(processGroup)
    result_part2 = subcols_part2.map(processGroup)
    result_part3 = subcols_part3.map(processGroup)
    result_part4 = subcols_part4.map(processGroup)

    # Combiner les résultats
    result_collections = ee.List(result_part1).cat(result_part2).cat(result_part3).cat(result_part4)
else:
    # Si le nombre de paires est inférieur ou égal à 4, appliquer processGroup directement
    result_collections = subs.map(processGroup)


In [106]:
# loop through each list of paired images
num_lists = subs.length().getInfo()
for i in range(0, num_lists):
    # determine the number of modis images between pairs
    num_imgs = ee.List(ee.List(subs.get(i)).get(2)).length()

    # determine the remainder of images, if not groups of 10
    remaining = num_imgs.mod(10)

    # create sequence of starting indices for modis images
    index_seq = ee.List.sequence(0, num_imgs.subtract(remaining), 10)

    # images to be grouped and predicted
    subList = ee.List(ee.List(subs.get(i)).get(2))

    # loop through indices predicting in batches of 10
    for x in range(0, index_seq.length().getInfo()):
        # starting index
        start = ee.Number(index_seq.get(x))

        # ending index
        end = ee.Algorithms.If(start.add(10).gt(num_imgs),
                               num_imgs,
                               start.add(10))

        # group of images to predict
        pred_group = subList.slice(start, end)
        landsat_t01 = ee.List(ee.List(subs.get(i)).get(0))
        modis_t01 = ee.List(ee.List(subs.get(i)).get(1))
        modis_tp = pred_group

        # get the start and end day values and year for the group to use
        # to label the file when exported to asset
        startDay = ee.Number.parse(ee.ImageCollection(pred_group)
                                   .first()
                                   .get('DOY'))
        endDay = ee.Number.parse(ee.ImageCollection(pred_group)
                                 .sort('system:time_start', False)
                                 .first()
                                 .get('DOY'))
        year = ee.Date(ee.ImageCollection(pred_group)
                       .sort('system:time_start', False)
                       .first()
                       .get('system:time_start')).format('Y')

        # start and end day of year
        doys = landsat_t01 \
            .map(lambda img: ee.String(ee.Image(img).get('DOY')).cat('_'))

        # register images
        landsat_t01, modis_t01, modis_tp = registerImages(landsat_t01,
                                                          modis_t01,
                                                          modis_tp)

        # prep landsat imagery (mask and format)
        maskedLandsat, pixPositions, pixBN = prepLandsat(landsat_t01,
                                                         kernel,
                                                         numPixels,
                                                         commonBandNames,
                                                         doys,
                                                         coverClasses)

        # prep modis imagery (mask and format)
        modSorted_t01, modSorted_tp = prepMODIS(modis_t01, modis_tp, kernel,
                                                numPixels, commonBandNames,
                                                pixBN)

        # calculate spectral distance
        specDist = calcSpecDist(maskedLandsat, modSorted_t01,
                                numPixels, pixPositions)

        # calculate spatial distance
        spatDist = calcSpatDist(pixPositions)

        # calculate weights from the spatial and spectral distances
        weights = calcWeight(spatDist, specDist)

        # calculate the conversion coefficients
        coeffs = calcConversionCoeff(maskedLandsat, modSorted_t01,
                                     doys, numPixels, commonBandNames)

        # predict all modis images in modis tp collection
        prediction = modSorted_tp \
            .map(lambda image:
                 predictLandsat(landsat_t01, modSorted_t01,
                                doys, ee.List(image),
                                weights, coeffs,
                                commonBandNames, numPixels))

        # create a list of new band names to apply to the multiband ndvi image
        # NOTE: cant export with names starting with 0
        preds = ee.ImageCollection(prediction).toBands()
        dates = modis_tp.map(lambda img:
                             ee.Image(img).get('system:time_start'))
        predNames = ee.List.sequence(0, prediction.length().subtract(1)) \
            .map(lambda i:
                 commonBandNames\
                     .map(lambda name:
                          ee.String(name)
                          .cat(ee.String(ee.Number(dates.get(i)).format()))))\
            .flatten()

        # export all predictions as a single multiband image
        # each band name corresponds to the timestamp for the image
        task = ee.batch.Export.image.toAsset(
            image=preds.rename(predNames).multiply(10000).toInt16(),
            description=ee.String(scene_name)
                        .cat(year)
                        .cat('_')
                        .cat(startDay.format())
                        .cat('_').cat(endDay.format()).getInfo(),
            assetId=ee.String(path)
                    .cat(ee.String(scene_name))
                    .cat(year)
                    .cat('_')
                    .cat(startDay.format())
                    .cat('_')
                    .cat(endDay.format()).getInfo(),
            region=ee.Image(prediction.get(0)).geometry(),
            scale=30)

        task.start()

In [109]:
task.status()

{'state': 'COMPLETED',
 'description': 'NDVI_2013_2013_250_253',
 'priority': 100,
 'creation_timestamp_ms': 1722974246744,
 'update_timestamp_ms': 1722976816527,
 'start_timestamp_ms': 1722976505319,
 'task_type': 'EXPORT_IMAGE',
 'destination_uris': ['https://code.earthengine.google.com/?asset=projects/ee-magniermorgane/assets/NDVI_2013_2013_250_253'],
 'attempt': 1,
 'batch_eecu_usage_seconds': 472.0864562988281,
 'id': 'O5GKQPBLF7GKH4DKQUNCTQYG',
 'name': 'projects/earthengine-legacy/operations/O5GKQPBLF7GKH4DKQUNCTQYG'}

In [104]:
predict_col = ee.ImageCollection(result_collections)
display(result_collections)

In [90]:
# Obtenir l'histogramme de l'image NDVI seuillée
'''import geemap.chart as chart

img = ee.Image(ee.ImageCollection(result_collections.get(0)).first())
#display(img)

my_sample = img.sample(region=region, numPixels=1000)

# Propriété de l'image pour l'histogramme
property = "ndvi"  # Remplacer "band_name" par le nom de la bande dans votre image

# Options pour l'histogramme
options = {
    "title": "ndvi histogram predictions",
    "xlabel": "Value",
    "ylabel": "Pixel count",
    "colors": ["#1d6b99"],
}

# Créer et afficher l'histogramme
histogram1 = chart.feature_histogram(my_sample, property, **options)
histogram1
'''


'import geemap.chart as chart\n\nimg = ee.Image(ee.ImageCollection(result_collections.get(0)).first())\n#display(img)\n\nmy_sample = img.sample(region=region, numPixels=1000)\n\n# Propriété de l\'image pour l\'histogramme\nproperty = "ndvi"  # Remplacer "band_name" par le nom de la bande dans votre image\n\n# Options pour l\'histogramme\noptions = {\n    "title": "ndvi histogram predictions",\n    "xlabel": "Value",\n    "ylabel": "Pixel count",\n    "colors": ["#1d6b99"],\n}\n\n# Créer et afficher l\'histogramme\nhistogram1 = chart.feature_histogram(my_sample, property, **options)\nhistogram1\n'

In [91]:
img = ee.Image(ee.ImageCollection(result_collections.get(0)).first())
#display(img)
#display(predictions_1)

In [92]:
ndviVis = {
    'min': -1,
    'max': 1,
    'palette' : [
        '#FFFFFF', '#E5F5FC', '#CCE6F4', '#B2DFF0', '#99D8EB', '#80D1E6',
        '#66CAE2', '#4DC3DD', '#33BCD9', '#1AB5D4', '#00AED0', '#00A7CB',
        '#009FC6', '#0098C2', '#0091BD', '#008AB8', '#0083B4', '#007CAF',
        '#0075AA', '#006EA6', '#0067A1', '#005F9D', '#005898', '#005193',
        '#004A8F', '#00438A', '#003C85', '#003481', '#002D7C', '#002677',
        '#001F72', '#00186D', '#001169', '#000A64', '#00035F', '#00005B'
    ]
}

m = geemap.Map()
m.centerObject(region,14)
m.addLayer(ee.Image(img.select('ndvi')),ndviVis,'img')
m

Map(center=[60.42909015971474, 17.23471649999347], controls=(WidgetControl(options=['position', 'transparent_b…

## NDVI median time series in wetlands

### Wetlands detection

#### Clouds filtering

In [93]:
def get_s2_sr_cld_col(roi, start_date, end_date):
    # Import and filter S2 SR.
    CLOUD_FILTER = 80 
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
        .filterBounds(roi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.calendarRange(5,9, 'month'))
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(roi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.calendarRange(4,10, 'month')))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

def add_cloud_bands(img):
    CLD_PRB_THRESH = 40

    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

def add_shadow_bands(img):

    NIR_DRK_THRESH = 0.15
    CLD_PRJ_DIST = 2

    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')))

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

def add_cld_shdw_mask(img):
    BUFFER = 100
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)

def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)

def count_pixels(image,roi): 
    pixel_count = image.select('B1').reduceRegion(
        reducer=ee.Reducer.count(),
        geometry=roi,
        scale=10,
        maxPixels=1e9
    ).get('B1')
    return image.set('pixel_count', pixel_count)


def get_clouds_free_collection_in_roi(roi,start_date, end_date,CLOUD_FILTER = 80,POURCENTAGE_MAX_CLOUDS = 80):
    
    s2_sr_cld_col = get_s2_sr_cld_col(roi, start_date, end_date)

    #s2_col_masked = s2_col.map(mask_s2_clouds).map(lambda image : image.clip(roi))

    s2_sr_masked = (s2_sr_cld_col.map(add_cld_shdw_mask)
                                .map(apply_cld_shdw_mask)).map(lambda image : image.clip(roi))

    # Calculate the threshold for pixel count (80% of the reference image)

    nb_pixels_time_series = s2_sr_masked.map(lambda image: count_pixels(image, roi))

    # Get the image with the maximum pixel count
    max_pixel_count_image = nb_pixels_time_series.sort('pixel_count', False).first()
    ref_img_pixel_count = max_pixel_count_image.get('pixel_count').getInfo()
    pixel_count_threshold = ref_img_pixel_count * 0.95

    # Filter the collection based on the pixel count threshold
    filtered_s2_sr_masked = nb_pixels_time_series.filter(ee.Filter.gte('pixel_count', pixel_count_threshold))

    return filtered_s2_sr_masked

#### Wetlands mask

In [94]:
def clustering(image,roi): 

    bands = image.bandNames() #If needs tp be changed

    training_samples = image.select(bands).sample(region=roi, scale=5, numPixels=15000, geometries=True)
    
    clusterer = ee.Clusterer.wekaKMeans(nClusters = 2, distanceFunction="Euclidean", maxIterations=75).train(training_samples)

    image_ndwi_clusters = image.addBands(image.cluster(clusterer)).select(['NDWI', 'cluster'])

    cluster = image.addBands(image.cluster(clusterer).rename('cluster'))

    return cluster

def water_classification(image, roi):
    
    image = clustering(image, roi)

    cluster_1 = image.updateMask(image.select('cluster').eq(1))
    cluster_2 = image.updateMask(image.select('cluster').eq(0))
    
    mean_1 = cluster_1.select('B1').reduceRegion(ee.Reducer.mean(),geometry = image.geometry(), scale = 10)
    mean_2 = cluster_2.select('B1').reduceRegion(ee.Reducer.mean(),geometry = image.geometry(), scale = 10)
    
    mean_1_value = mean_1.getNumber('B1')
    mean_2_value = mean_2.getNumber('B1')
    
    #display(mean_1_value)
    #display(mean_2_value)
    
    water = ee.Image(ee.Algorithms.If(mean_1_value.gt(mean_2_value), image.select('cluster').eq(0), image.select('cluster').eq(1)))

    return water

def apply_mask(image,wetlands_mask):
    image = image.updateMask(wetlands_mask)
    return image

import eemont
def getWetlandsS2(roi, min_water_date, max_water_date):
    
    flooded_img = get_clouds_free_collection_in_roi(roi, max_water_date, max_water_date.advance(1,'day')).spectralIndices('water').first().resample('bicubic').clip(roi)
    dry_img = get_clouds_free_collection_in_roi(roi, min_water_date, min_water_date.advance(1,'day')).spectralIndices('water').first().resample('bicubic').clip(roi)

    flooded_mask = water_classification(flooded_img, roi)
    dry_mask = water_classification(dry_img, roi)

    wetlands_mask = flooded_mask.subtract(dry_mask)

    return wetlands_mask

In [95]:
roi = region
min_water_date = ee.Date('2022-09-25')
max_water_date = ee.Date('2018-05-09')
wetlands = getWetlandsS2(roi, min_water_date, max_water_date)

In [96]:
#rgbVis = {'bands': ['B4','B3', 'B2'], 'min' : 0 , 'max' : 0.1}
m = geemap.Map()
m.centerObject(region,14)
m.addLayer(wetlands.selfMask(), {'palette': ['blue']}, 'mask')
#m

In [97]:
predictions_wetlands_0 = predict_col.map(lambda image : apply_mask(image,wetlands))
#predictions_wetlands_1 = predictions_1.map(lambda image : apply_mask(image,wetlands))

display(predictions_wetlands_0)
#display(predictions_wetlands_1)

### Time series

In [101]:
def calcul_median_ndvi(image):
    ndvi_median = image.reduceRegion(
        reducer=ee.Reducer.median(),
        geometry= region,
        scale=30,
        maxPixels=1e9
    )
    return ee.Feature(None, ndvi_median).copyProperties(image)

ndvi_ts_0 = predict_col.map(calcul_median_ndvi)
#ndvi_ts_1 = predictions_wetlands_1.map(calcul_median_ndvi)
'''
ndvi_ts_2 = predictions_2.map(calcul_median_ndvi)
ndvi_ts_3 = predictions_3.map(calcul_median_ndvi)
ndvi_ts_4 = predictions_4.map(calcul_median_ndvi)

ndvi_ts_5 = predictions_5.map(calcul_median_ndvi)

ndvi_ts_6 = predictions_6.map(calcul_median_ndvi)
ndvi_ts_7 = predictions_7.map(calcul_median_ndvi)
ndvi_ts_8 = predictions_8.map(calcul_median_ndvi)
ndvi_ts_9 = predictions_9.map(calcul_median_ndvi)
ndvi_ts_10 = predictions_10.map(calcul_median_ndvi)
ndvi_ts_11 = predictions_11.map(calcul_median_ndvi)
'''

'\nndvi_ts_2 = predictions_2.map(calcul_median_ndvi)\nndvi_ts_3 = predictions_3.map(calcul_median_ndvi)\nndvi_ts_4 = predictions_4.map(calcul_median_ndvi)\n\nndvi_ts_5 = predictions_5.map(calcul_median_ndvi)\n\nndvi_ts_6 = predictions_6.map(calcul_median_ndvi)\nndvi_ts_7 = predictions_7.map(calcul_median_ndvi)\nndvi_ts_8 = predictions_8.map(calcul_median_ndvi)\nndvi_ts_9 = predictions_9.map(calcul_median_ndvi)\nndvi_ts_10 = predictions_10.map(calcul_median_ndvi)\nndvi_ts_11 = predictions_11.map(calcul_median_ndvi)\n'

In [102]:
display(ndvi_ts_0)
#display(ndvi_ts_1)

In [100]:
ndvi_0_df = ee.data.computeFeatures({'expression': ndvi_ts_0, 'fileFormat': 'PANDAS_DATAFRAME'})
#ndvi_1_df = ee.data.computeFeatures({'expression': ndvi_ts_1, 'fileFormat': 'PANDAS_DATAFRAME'})

'''
ndvi_2_df = ee.data.computeFeatures({'expression': ndvi_ts_2, 'fileFormat': 'PANDAS_DATAFRAME'})
ndvi_3_df = ee.data.computeFeatures({'expression': ndvi_ts_3, 'fileFormat': 'PANDAS_DATAFRAME'})
ndvi_4_df = ee.data.computeFeatures({'expression': ndvi_ts_4, 'fileFormat': 'PANDAS_DATAFRAME'})
ndvi_5_df = ee.data.computeFeatures({'expression': ndvi_ts_5, 'fileFormat': 'PANDAS_DATAFRAME'})
ndvi_6_df = ee.data.computeFeatures({'expression': ndvi_ts_6, 'fileFormat': 'PANDAS_DATAFRAME'})
ndvi_7_df = ee.data.computeFeatures({'expression': ndvi_ts_7, 'fileFormat': 'PANDAS_DATAFRAME'})
ndvi_8_df = ee.data.computeFeatures({'expression': ndvi_ts_8, 'fileFormat': 'PANDAS_DATAFRAME'})
ndvi_9_df = ee.data.computeFeatures({'expression': ndvi_ts_9, 'fileFormat': 'PANDAS_DATAFRAME'})
ndvi_10_df = ee.data.computeFeatures({'expression': ndvi_ts_10, 'fileFormat': 'PANDAS_DATAFRAME'})
ndvi_11_df = ee.data.computeFeatures({'expression': ndvi_ts_11, 'fileFormat': 'PANDAS_DATAFRAME'})
'''

EEException: ImageCollection.fromImages, argument 'images': Invalid type.
Expected type: List<Image<unknown bands>>.
Actual type: List<ImageCollection>.
Actual value: [<ImageCollection>, <ImageCollection>, <ImageCollection>]

In [ ]:
ndvi_0_df.head()

,geo,DOY
0,None,69
1,None,70
2,None,71
3,None,72
4,None,73


In [ ]:
import pandas as pd
ndvi_array = [ndvi_0_df, ndvi_1_df]#, ndvi_2_df, ndvi_3_df, ndvi_4_df]
       #ndvi_5_df, ndvi_6_df, ndvi_7_df, ndvi_8_df, ndvi_9_df,
       #ndvi_10_df, ndvi_11_df]

# Fusionner les DataFrames
ndvi_df = pd.concat(ndvi_array, ignore_index=True)
ndvi_df['DOY'] = ndvi_df['DOY'].astype(int)
ndvi_df = ndvi_df.sort_values(by='DOY')

import plotly.express as px

fig = px.scatter(ndvi_df, x='DOY', y='ndvi', title='NDVI Time Series', labels={'Date': 'Date', 'NDVI': 'NDVI'})

# Afficher le graphique
fig.show()

NameError: name 'ndvi_1_df' is not defined

#### SAVE 

In [ ]:
ndvi_df.to_csv('ndvi_modis_l8_fusion_2015_in_wetlands.csv', index=False)

## NDVI density time series

In [ ]:
hn_min = 0 
hn_max = 1
hn = 0.1 
ndvi_density_0 = predictions_0.map(lambda image : geemap.create_contours(image.select('ndvi'), hn_min, hn_max, hn, region=None))
ndvi_density_1 = predictions_1.map(lambda image : geemap.create_contours(image.select('ndvi'), hn_min, hn_max, hn, region=None))

display(ndvi_density_0)
display(ndvi_density_1)

In [ ]:
hn_min = 0
hn_max = 1
hn = 0.15
density_vis = {
  'min': hn_min,
  'max': hn_max,
  'palette': ['0000ff','00ffff','ffff00','ff0000','ffffff']}

color = ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
               '74A901', '66A000', '529400', '3E8601', '207401', '056201',
               '004C00', '023B01', '012E01', '011D01', '011301']
palette = {"min":0, "max":1, 'palette':color}
image = predictions_0.first()
# initialize our map
ma = geemap.Map()
ma.centerObject(roi, 14)
ma.addLayer(image.select('ndvi'), palette, "NDVI")

ndvi_contours = geemap.create_contours(image.select('ndvi'), hn_min, hn_max, hn, region=None)
ma.addLayer(ndvi_contours, {'palette': ['0000ff', '00ffff', 'ffff00', 'ff0000', 'ffffff']}, 'contours', True)
ma.add_colorbar(density_vis, label="NDVI density", layer_name="NDVI", orientation="vertical", transparent_bg=True,)
ma

Map(center=[60.42909015971474, 17.23471649999347], controls=(WidgetControl(options=['position', 'transparent_b…

### Clustering

In [ ]:
def density_ndvi_clustering(image, nClusters):
    training = image.sample(
    region = roi,
    scale = 10,
    numPixels = 1000,
    geometries=True
    )

    kmeans = ee.Clusterer.wekaKMeans(nClusters = nClusters, init = 2, distanceFunction = 'Euclidian', maxIterations = 500).train(training)
    kmean_rs = image.cluster(kmeans)
    return image.addBands(kmean_rs)

ndvi_clusters_0 = ndvi_density_0.map(lambda image : density_ndvi_clustering(image,10))
ndvi_clusters_1 = ndvi_density_1.map(lambda image : density_ndvi_clustering(image,10))

display(ndvi_clusters_0)
#display(ndvi_clusters_1)

In [ ]:
image = ndvi_density_1.first()
training = image.sample(
    region = roi,
    scale = 10,
    numPixels = 1000,
    geometries=True
    )

kmeans = ee.Clusterer.wekaKMeans(nClusters = 10, init = 2, distanceFunction = 'Euclidian', maxIterations = 500).train(training)
kmean_rs = image.cluster(kmeans)
display(kmean_rs)

### Time series

In [ ]:
def calcul_surface_and_ndvi_clusters(image):
    # Récupérer les clusters de l'image
    clusters = image.select('cluster')
    ndvi = image.select('NDVI')
    
    # Calculer la surface de chaque cluster
    cluster_area = clusters.multiply(ee.Image.pixelArea()).reduceRegion(
        reducer=ee.Reducer.sum().group(groupField=1, groupName='cluster'),
        geometry=region,
        scale=30,
        maxPixels=1e9
    )
    
    # Calculer la valeur NDVI médiane pour chaque cluster
    cluster_ndvi = ndvi.reduceRegion(
        reducer=ee.Reducer.median().group(groupField=1, groupName='cluster'),
        geometry=region,
        scale=30,
        maxPixels=1e9
    )
    
    # Extraire les valeurs des résultats de surface
    cluster_area_dict = ee.Dictionary(cluster_area.get('groups'))
    cluster_area_list = cluster_area_dict.map(lambda k, v: ee.Number(ee.Dictionary(v).get('sum')))
    
    # Extraire les valeurs des résultats de NDVI
    cluster_ndvi_dict = ee.Dictionary(cluster_ndvi.get('groups'))
    cluster_ndvi_list = cluster_ndvi_dict.map(lambda k, v: ee.Number(ee.Dictionary(v).get('median')))
    
    # Créer une feature avec les surfaces et NDVI des clusters
    feature = ee.Feature(None)
    
    # Ajouter les surfaces des clusters à la feature
    cluster_keys = cluster_area_list.keys()
    cluster_keys.map(lambda key: feature.set(ee.String('surface_cluster_').cat(key), cluster_area_list.get(key)))
    
    # Ajouter les NDVI médianes des clusters à la feature
    cluster_keys.map(lambda key: feature.set(ee.String('ndvi_cluster_').cat(key), cluster_ndvi_list.get(key)))
    
    # Ajouter le DOY à la feature
    feature = feature.set('DOY', image.get('DOY'))
    
    return feature

# Appliquer la fonction aux collections d'images
ndvi_ts_0 = ndvi_clusters_0.map(calcul_surface_and_ndvi_clusters)
ndvi_ts_1 = ndvi_clusters_1.map(calcul_surface_and_ndvi_clusters)

'\nndvi_ts_2 = predictions_2.map(calcul_median_ndvi)\nndvi_ts_3 = predictions_3.map(calcul_median_ndvi)\nndvi_ts_4 = predictions_4.map(calcul_median_ndvi)\n\nndvi_ts_5 = predictions_5.map(calcul_median_ndvi)\n\nndvi_ts_6 = predictions_6.map(calcul_median_ndvi)\nndvi_ts_7 = predictions_7.map(calcul_median_ndvi)\nndvi_ts_8 = predictions_8.map(calcul_median_ndvi)\nndvi_ts_9 = predictions_9.map(calcul_median_ndvi)\nndvi_ts_10 = predictions_10.map(calcul_median_ndvi)\nndvi_ts_11 = predictions_11.map(calcul_median_ndvi)\n'

In [ ]:
display(ndvi_ts_0)
display(ndvi_ts_1)

In [ ]:
ndvi_0_df = ee.data.computeFeatures({'expression': ndvi_ts_0, 'fileFormat': 'PANDAS_DATAFRAME'})
ndvi_1_df = ee.data.computeFeatures({'expression': ndvi_ts_1, 'fileFormat': 'PANDAS_DATAFRAME'})

'''
ndvi_2_df = ee.data.computeFeatures({'expression': ndvi_ts_2, 'fileFormat': 'PANDAS_DATAFRAME'})
ndvi_3_df = ee.data.computeFeatures({'expression': ndvi_ts_3, 'fileFormat': 'PANDAS_DATAFRAME'})
ndvi_4_df = ee.data.computeFeatures({'expression': ndvi_ts_4, 'fileFormat': 'PANDAS_DATAFRAME'})
ndvi_5_df = ee.data.computeFeatures({'expression': ndvi_ts_5, 'fileFormat': 'PANDAS_DATAFRAME'})
ndvi_6_df = ee.data.computeFeatures({'expression': ndvi_ts_6, 'fileFormat': 'PANDAS_DATAFRAME'})
ndvi_7_df = ee.data.computeFeatures({'expression': ndvi_ts_7, 'fileFormat': 'PANDAS_DATAFRAME'})
ndvi_8_df = ee.data.computeFeatures({'expression': ndvi_ts_8, 'fileFormat': 'PANDAS_DATAFRAME'})
ndvi_9_df = ee.data.computeFeatures({'expression': ndvi_ts_9, 'fileFormat': 'PANDAS_DATAFRAME'})
ndvi_10_df = ee.data.computeFeatures({'expression': ndvi_ts_10, 'fileFormat': 'PANDAS_DATAFRAME'})
ndvi_11_df = ee.data.computeFeatures({'expression': ndvi_ts_11, 'fileFormat': 'PANDAS_DATAFRAME'})
'''

In [ ]:
import pandas as pd
ndvi_array = [ndvi_0_df, ndvi_1_df]#, ndvi_2_df, ndvi_3_df, ndvi_4_df]
       #ndvi_5_df, ndvi_6_df, ndvi_7_df, ndvi_8_df, ndvi_9_df,
       #ndvi_10_df, ndvi_11_df]

# Fusionner les DataFrames
ndvi_df = pd.concat(ndvi_array, ignore_index=True)

In [ ]:
ndvi_df['DOY'] = ndvi_df['DOY'].astype(int)
ndvi_df = ndvi_df.sort_values(by='DOY')

In [ ]:
import plotly.express as px

fig = px.scatter(ndvi_df, x='DOY', y='ndvi', title='NDVI Time Series', labels={'Date': 'Date', 'NDVI': 'NDVI'})

# Afficher le graphique
fig.show()

#### SAVE

In [ ]:
ndvi_df.to_csv('ndvi_modis_l8_fusion_2013_in_wetlands.csv', index=False)